# Import

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import os

from sysidentpy.model_structure_selection import FROLS 
from sysidentpy.basis_function._basis_function import Polynomial 
from sysidentpy.metrics import root_relative_squared_error 
from sysidentpy.utils.display_results import results
from sysidentpy.utils.plotting import plot_residues_correlation, plot_results

In [2]:
dirname = os.getcwd()
filename = dirname + "/data/pump_station_data.parquet"
print(filename)

/home/smrobot/Industrial-IoT-For-Digitization-Of-Electronic-Assets-Course/Final Project/data/pump_station_data.parquet


In [ ]:
pump_station_data = pd.read_parquet(filename)
pump_station_data.head(5)

train = pump_station_data.loc["2023-01-27":"2023-01-30"]
test = pump_station_data.loc["2023-02-27":"2023-03-02"] 

#Consider resample()
pump_station_data = pump_station_data.resample("1T").mean()

pump_station_data = pump_station_data[(pump_station_data["pump1_rpm"] > 0) & (pump_station_data["pump4_rpm"] == 0)]
x_train = train["pump1_rpm"].values.reshape(-1,1)
y_train = train["outflow"].values.reshape(-1,1)
x_test = test["pump1_rpm"].values.reshape(-1,1)
y_test = test["outflow"].values.reshape(-1,1)

In [ ]:
basis_function = Polynomial(degree=1)

model = FROLS(
    order_selection=True,
    n_info_values=20,
    extended_least_squares=False,
    ylag=6,
    xlag=6,
    info_criteria="aic",
    estimator="least_squares",
    basis_function=basis_function,
)
model.fit(X=x_train, 
          y=y_train)

In [ ]:
yhat = model.predict(X=x_test, 
                     y=y_test,
                     forecast_horizon=5)

In [ ]:
residuals = y_test - yhat

# Plotting the histogram of residuals
plt.hist(residuals, bins=100, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
#plt.xlim(-50,50)
plt.show()